In [52]:
%matplotlib inline
import numpy as np
import matplotlib as plt
import seaborn as sns
from scipy.stats import norm
import math

def monte_carlo(k, sigma):
    P_list = []
    for Z in epsilons:
        S_T = S_0*np.exp((r-0.5*(sigma**2))*T+sigma*np.sqrt(T)*Z)
        P = max((k - S_T), 0)
        P_list.append(P)

    P_est = np.exp(-r*T)*np.mean(P_list)
    PS_discount = [np.exp(-r*T)*i for i in Ps]
    S_E = np.std(PS_discount) / np.sqrt(N)
    
#     S_E = np.std(P_list) / np.sqrt(N)
    higer_confidence_intrval_95 = P_est + (S_E * 1.96)
    lower_confidence_intrval_95 = P_est - (S_E * 1.96)
    return P_est, S_E, higer_confidence_intrval_95, lower_confidence_intrval_95

def monte_carlo_bump(k, sigma, e):
    Ps = []
    Ps_bump = []
    Zs = np.random.randn(int(N))

    for Z in Zs:
        
        S_T = S_0 * np.exp((r - 0.5*(sigma**2)) * T + sigma * np.sqrt(T) * Z)
        S_T_bump = (S_0+e) * np.exp((r - 0.5*(sigma**2)) * T + sigma * np.sqrt(T) * Z)
        
        P = max((k - S_T), 0)
        P_bump =  max((k - S_T_bump), 0)
        
        Ps_bump.append(P_bump)
        Ps.append(P)
    
    P_est = np.exp(-r*T) * np.mean(Ps)
    PS_discount = [np.exp(-r*T)*i for i in Ps]
    SE_est = np.std(PS_discount) / np.sqrt(N)

    P_bump_est = np.exp(-r*T) * np.mean(Ps_bump)
    PS_discount_bump = [np.exp(-r*T)*i for i in Ps]

    SE_bump_est = np.std(PS_discount_bump) / np.sqrt(N)
    
    delta = (P_bump_est - P_est) / e
    
    var_P = np.var(Ps)
    var_P_bump = np.var(Ps_bump)
    cov = np.cov(Ps, Ps_bump)[0][1]
    var_delta = (.5*e**2)*(var_P + var_P_bump - 2*cov)

    return delta, P_est, SE_est, var_delta

def monte_carlo_bump_2s(k, sigma, e):
    
    Ps = []
    Ps_bump = []
    Zs = np.random.randn(int(N))
    Zs_2 = np.random.randn(int(N))

    for Z1, Z2 in zip(Zs,Zs_2):
        
        S_T = S_0*np.exp((r - 0.5*(sigma**2))*T+sigma*np.sqrt(T)*Z1)
        S_T_bump = (S_0+e)*np.exp((r - 0.5*(sigma**2))*T+sigma*np.sqrt(T)*Z2)
        
        P = max((k - S_T), 0)
        P_bump =  max((k - S_T_bump), 0)
        
        Ps_bump.append(P_bump)
        Ps.append(P)
    
    P_est = np.exp(-r*T) * np.mean(Ps)
    PS_discount = [np.exp(-r*T)*i for i in Ps]
    SE_est = np.std(PS_discount) / np.sqrt(N)

    P_bump_est = np.exp(-r*T) * np.mean(Ps_bump)
    PS_discount_bump = [np.exp(-r*T)*i for i in Ps]
    SE_bump_est = np.std(PS_discount_bump) / np.sqrt(N)
    
    delta = (P_bump_est - P_est)/e
    var_P = np.var(Ps)
    var_P_bump = np.var(Ps_bump)
    cov = np.cov(Ps, Ps_bump)[0][1]
    var_delta = (.5*e**2)*(var_P + var_P_bump - 2*cov)    
    
    return delta, P_est, SE_est, var_delta

def BS_delta():
    t = 0
    d1 = (np.log(S_0/K) + (r + 0.5*sigma**2) * (T - t)) / (sigma*np.sqrt(T - t))
    BS_delta = norm.cdf(d1)-1
    return BS_delta

def BS_C():
    d1 = (np.log(S_0/K)+(r+sigma**2/2) * T) / sigma*np.sqrt(T)
    d2 = d1 - sigma*T
    c = norm.cdf(d1)*S_0-norm.cdf(d2)*K*np.exp(-1*r*T)
    return c

def BS_P():
    d1 = (np.log(S_0/K)+(r+sigma**2/2) * T) / sigma*np.sqrt(T)
    d2 = d1 - sigma*T
    p = norm.cdf(-d2)*K*np.exp(-r*T) - norm.cdf(-d1)*S_0
    return p
def BS_P_sigma(sigma):
    d1 = (np.log(S_0/K)+(r+sigma**2/2) * T) / sigma*np.sqrt(T)
    d2 = d1 - sigma*T
    p = norm.cdf(-d2)*K*np.exp(-r*T) - norm.cdf(-d1)*S_0
    return p

def BS_P_strike(K):
    d1 = (np.log(S_0/K)+(r+sigma**2/2) * T) / sigma*np.sqrt(T)
    d2 = d1 - sigma*T
    p = norm.cdf(-d2)*K*np.exp(-r*T) - norm.cdf(-d1)*S_0
    return p

In [3]:
N = 100000
S_0 = 100
r= 0.06
sigma = 0.2
K = 99
T = 1

repetitions = 20

In [4]:
results = {}

In [4]:
results['trials'] = {}
Ns = np.logspace(4, 7, 50, endpoint=True)

results['trials']['N'] = []
results['trials']['p'] = []
results['trials']['SE'] = []
results['trials']['CI_H'] = []
results['trials']['CI_L'] = []

for i in range(repetitions):
    results['trials']['N'].append([])
    results['trials']['p'].append([])
    results['trials']['SE'].append([])
    results['trials']['CI_H'].append([])
    results['trials']['CI_L'].append([])
    print(i)
    for N in Ns:
        epsilons = np.random.randn(int(N))
        P_est, S_E, higer_confidence_intrval_95, lower_confidence_intrval_95 = monte_carlo(K, sigma)

        results['trials']['N'][i].append(N)
        results['trials']['p'][i].append(P_est)
        results['trials']['SE'][i].append(S_E)
        results['trials']['CI_H'].append(higer_confidence_intrval_95)
        results['trials']['CI_L'].append(lower_confidence_intrval_95)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [6]:
N = 100000
epsilons = np.random.randn(N)
S_0 = 100
r= 0.06
sigma = 0.2
K = 99
T = 1

In [17]:
results['strike'] = {}
Ks = np.linspace(1 , 200, 30)

results['strike']['N'] = []
results['strike']['p'] = []
results['strike']['SE'] = []
results['strike']['CI_H'] = []
results['strike']['CI_L'] = []
results['strike']['BS_p'] = []

for i in range(repetitions):
    results['strike']['N'].append([])
    results['strike']['p'].append([])
    results['strike']['SE'].append([])
    results['strike']['CI_H'].append([])
    results['strike']['CI_L'].append([])
    results['strike']['BS_p'].append([])
    print(i)
    for K in Ks:
        epsilons = np.random.randn(int(N))
        P_est, S_E, higer_confidence_intrval_95, lower_confidence_intrval_95 = monte_carlo(K, sigma)

        results['strike']['N'][i].append(N)
        results['strike']['p'][i].append(P_est)
        results['strike']['SE'][i].append(S_E)
        results['strike']['CI_H'].append(higer_confidence_intrval_95)
        results['strike']['CI_L'].append(lower_confidence_intrval_95)
        results['strike']['BS_p'][i].append(BS_P_strike(K))        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [7]:
N = 100000
epsilons = np.random.randn(N)
S_0 = 100
r= 0.06
sigma = 0.2
K = 99
T = 1

In [8]:
results['sigma'] = {}
sigmas = np.linspace(0.01 , 0.99, 50)

results['sigma']['N'] = []
results['sigma']['p'] = []
results['sigma']['SE'] = []
results['sigma']['CI_H'] = []
results['sigma']['CI_L'] = []
results['sigma']['BS_p'] = []

for i in range(repetitions):
    results['sigma']['N'].append([])
    results['sigma']['p'].append([])
    results['sigma']['SE'].append([])
    results['sigma']['CI_H'].append([])
    results['sigma']['CI_L'].append([])
    results['sigma']['BS_p'].append([])
    print(i)
    for sigma in sigmas:
        P_est, S_E, higer_confidence_intrval_95, lower_confidence_intrval_95 = monte_carlo(K, sigma)

        results['sigma']['N'][i].append(N)
        results['sigma']['p'][i].append(P_est)
        results['sigma']['SE'][i].append(S_E)
        results['sigma']['CI_H'].append(higer_confidence_intrval_95)
        results['sigma']['CI_L'].append(lower_confidence_intrval_95)
        results['sigma']['BS_p'][i].append(BS_P_sigma(sigma))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [47]:
N = 100000
epsilons = np.random.randn(N)
S_0 = 100
r= 0.06
sigma = 0.2
K = 99
T = 1

Ks = [50, 75, 100]
sigmas = list(np.linspace(0.01 , 0.99, 20))
# print(sigmas)
# repetitions = 20

BS_p = BS_P()

print(BS_p)
# repetitions = 1

results['MC_strike_volatility'] = {}
results['MC_strike_volatility']['Ks'] = Ks
results['MC_strike_volatility']['sigmas'] = sigmas
for K in Ks:    
    print('K:' + str(K))
    results['MC_strike_volatility'][str(K)] = {}
    results['MC_strike_volatility'][str(K)]['p'] = []
    results['MC_strike_volatility'][str(K)]['SE'] = []
    for sigma in sigmas:
        P_est, S_E, higer_confidence_intrval_95, lower_confidence_intrval_95 = monte_carlo(K, sigma)
        results['MC_strike_volatility'][str(K)]['p'].append(P_est)
        results['MC_strike_volatility'][str(K)]['SE'].append(S_E)
with open('results_strike_volatility.json', 'w') as fp:
    json.dump(results, fp)

[0.01, 0.06157894736842105, 0.1131578947368421, 0.16473684210526315, 0.2163157894736842, 0.26789473684210524, 0.3194736842105263, 0.37105263157894736, 0.4226315789473684, 0.47421052631578947, 0.5257894736842105, 0.5773684210526315, 0.6289473684210526, 0.6805263157894736, 0.7321052631578947, 0.7836842105263158, 0.8352631578947368, 0.8868421052631579, 0.9384210526315789, 0.99]
4.778969051891714
K:50
K:75
K:100


In [49]:
N = 100000
epsilons = np.random.randn(N)
S_0 = 100
r= 0.06
sigma = 0.2
K = 99
T = 1

# Ks = [50, 75, 100]
# sigmas = list(np.linspace(0.01 , 0.99, 20))
Ks = list(np.linspace(50, 150, 20))
sigmas = [0.1 , 0.3, 0.5]
# print(sigmas)
# repetitions = 20

BS_p = BS_P()

print(BS_p)
# repetitions = 1

results['MC_strike_volatility2'] = {}
results['MC_strike_volatility2']['Ks'] = Ks
results['MC_strike_volatility2']['sigmas'] = sigmas
for sigma in sigmas:    
    print('K:' + str(sigma))
    results['MC_strike_volatility2'][str(sigma)] = {}
    results['MC_strike_volatility2'][str(sigma)]['p'] = []
    results['MC_strike_volatility2'][str(sigma)]['SE'] = []
    for K in Ks:
        P_est, S_E, higer_confidence_intrval_95, lower_confidence_intrval_95 = monte_carlo(K, sigma)
        results['MC_strike_volatility2'][str(sigma)]['p'].append(P_est)
        results['MC_strike_volatility2'][str(sigma)]['SE'].append(S_E)
with open('MC_strike_volatility2.json', 'w') as fp:
    json.dump(results, fp)

4.778969051891714
K:0.1
K:0.3
K:0.5


In [9]:
N = 100000
epsilons = np.random.randn(N)
S_0 = 100
r= 0.06
sigma = 0.2
K = 99
T = 1

In [11]:
results['MC_bump'] = {}
Ns = np.logspace(4, 7, 50, endpoint=True)
BS_d = BS_delta()

e = 0.01

results['MC_bump']['N'] = []
results['MC_bump']['delta'] = []
results['MC_bump']['p'] = []
results['MC_bump']['SE'] = []
results['MC_bump']['CI_H'] = []
results['MC_bump']['CI_L'] = []
results['MC_bump']['var_delta'] = []
for i in range(repetitions):
    results['MC_bump']['N'].append([])
    results['MC_bump']['delta'].append([])
    results['MC_bump']['p'].append([])
    results['MC_bump']['SE'].append([])
    results['MC_bump']['var_delta'].append([])
    results['MC_bump']['CI_H'].append([])
    results['MC_bump']['CI_L'].append([])
    for N in Ns:    

        MC_delta, P_est, SE_est, var_delta = monte_carlo_bump(K, sigma, e)
        relative_error = np.abs(MC_delta - BS_d)/BS_d

        results['MC_bump']['N'][i].append(N)
        results['MC_bump']['delta'][i].append(MC_delta)
        results['MC_bump']['p'][i].append(P_est)
        results['MC_bump']['SE'][i].append(SE_est)
        results['MC_bump']['var_delta'][i].append(var_delta)  
        results['MC_bump']['CI_H'].append(higer_confidence_intrval_95)
        results['MC_bump']['CI_L'].append(lower_confidence_intrval_95)

In [54]:
N = 100000
epsilons = np.random.randn(N)
S_0 = 100
r= 0.06
sigma = 0.2
K = 99
T = 1

In [13]:
results['MC_bump_2s'] = {}
Ns = np.logspace(4, 7, 50, endpoint=True)
BS_d = BS_delta()

e = 0.01

results['MC_bump_2s']['N'] = []
results['MC_bump_2s']['delta'] = []
results['MC_bump_2s']['p'] = []
results['MC_bump_2s']['SE'] = []
results['MC_bump_2s']['CI_H'] = []
results['MC_bump_2s']['CI_L'] = []
results['MC_bump_2s']['var_delta'] = []
for i in range(repetitions):
    results['MC_bump_2s']['N'].append([])
    results['MC_bump_2s']['delta'].append([])
    results['MC_bump_2s']['p'].append([])
    results['MC_bump_2s']['SE'].append([])
    results['MC_bump_2s']['var_delta'].append([])
    results['MC_bump_2s']['CI_H'].append([])
    results['MC_bump_2s']['CI_L'].append([])
    for N in Ns:    

        MC_delta, P_est, SE_est, var_delta = monte_carlo_bump_2s(K, sigma, e)
        relative_error = np.abs(MC_delta - BS_d)/BS_d

        results['MC_bump_2s']['N'][i].append(N)
        results['MC_bump_2s']['delta'][i].append(MC_delta)
        results['MC_bump_2s']['p'][i].append(P_est)
        results['MC_bump_2s']['SE'][i].append(SE_est)
        results['MC_bump_2s']['var_delta'][i].append(var_delta)  
        results['MC_bump_2s']['CI_H'].append(higer_confidence_intrval_95)
        results['MC_bump_2s']['CI_L'].append(lower_confidence_intrval_95)
        

In [ ]:
import json
import datetime
now = datetime.datetime.now()
with open('resultsA2_'+str(now.day)+str(now.hour)+'.json', 'w') as fp:
    json.dump(results, fp)

In [20]:
BS_P()

4.778969051891714

In [43]:
results = {}
results['MC_bump_epsilons'] = {}
Ns = [10**4, 10**5, 10**6]
bumps = [0.01, 0.05 ,0.1]

BS_d = BS_delta()
# N = 100000
S_0 = 100
r= 0.06
sigma = 0.2
K = 99
T = 1

repetitions = 20
# repetitions = 1

results['MC_bump_epsilons']['N'] = Ns

for N in Ns:    
    print('N:' + str(N))
    results['MC_bump_epsilons'][str(N)] = {}
    for e in bumps:
        results['MC_bump_epsilons'][str(N)][str(e)] = {}
        results['MC_bump_epsilons'][str(N)][str(e)]['delta'] = []
        results['MC_bump_epsilons'][str(N)][str(e)]['RE'] = []
        for i in range(repetitions):

#             print('e:' + str(e))
            MC_delta, P_est, SE_est, var_delta = monte_carlo_bump(K, sigma, e)
            relative_error = np.abs(MC_delta - BS_d)/BS_d
            results['MC_bump_epsilons'][str(N)][str(e)]['delta'].append(MC_delta)
            results['MC_bump_epsilons'][str(N)][str(e)]['RE'].append(relative_error)
        
with open('MC_bump_epsilons.json', 'w') as fp:
    json.dump(results, fp)

N:10000
N:100000
N:1000000


In [48]:
results = {}
results['MC_bump_2s_epsilons'] = {}
Ns = [10**4, 10**5, 10**6]
bumps = [0.01, 0.05 ,0.1]

BS_d = BS_delta()
# N = 100000
S_0 = 100
r= 0.06
sigma = 0.2
K = 99
T = 1

repetitions = 20
# repetitions = 1

results['MC_bump_2s_epsilons']['N'] = Ns

for N in Ns:    
    print('N:' + str(N))
    results['MC_bump_2s_epsilons'][str(N)] = {}
    for e in bumps:
        results['MC_bump_2s_epsilons'][str(N)][str(e)] = {}
        results['MC_bump_2s_epsilons'][str(N)][str(e)]['delta'] = []
        results['MC_bump_2s_epsilons'][str(N)][str(e)]['RE'] = []
        for i in range(repetitions):

#             print('e:' + str(e))
            MC_delta, P_est, SE_est, var_delta = monte_carlo_bump_2s(K, sigma, e)
            relative_error = np.abs(MC_delta - BS_d)/BS_d
            results['MC_bump_2s_epsilons'][str(N)][str(e)]['delta'].append(MC_delta)
            results['MC_bump_2s_epsilons'][str(N)][str(e)]['RE'].append(relative_error)
        
with open('MC_bump_2s_epsilons.json', 'w') as fp:
    json.dump(results, fp)

N:10000
N:100000
N:1000000


# Part III

In [59]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  6 12:17:21 2019

@author: Thies
"""

## Import packages
import numpy as np
from scipy.stats import norm

## Choose parameter values
S_0 = 100
r = 0.06
sigma = 0.2
K = 99
T = 1
N = 30
R = 100000
delta = T / N

def Generate_Price_Sequence_Euler(epsilons):    
    S = []
    S.append(S_0)
    for i,e in enumerate(epsilons):
        St = S[i] + r * S[i] * delta + sigma * S[i] * e * np.sqrt(delta)
        S.append(St)
    return S[1:]

def Generate_Price_Sequence_Milstein(epsilons):    
    S = []
    S.append(S_0)
    for i,e in enumerate(epsilons):
        St = S[i] * (1 + (r - 0.5 * sigma**2) * delta + sigma * e * np.sqrt(delta) + 0.5 * sigma**2 * e**2 * delta)
        S.append(St)
    return S[1:]

def Calculate_Value_Asian_Call(S):
    A = np.mean(S)
    C = max((A - K), 0)
    return C

def MC_Asian_Call():
    C_list = []
    
    for i in range(0,R):
        epsilons = np.random.randn(N)
        S = Generate_Price_Sequence_Milstein(epsilons)
        C = Calculate_Value_Asian_Call(S)
        C_list.append(C)
    return C_list

def Antithetic_MC_Asian_Call():
    C_list = []
    
    for i in range(0,R):
        epsilons = np.random.randn(N)
        S_Plus = Generate_Price_Sequence_Milstein(epsilons)
        C_Plus = Calculate_Value_Asian_Call(S_Plus)
        S_Minus = Generate_Price_Sequence_Milstein(-epsilons)
        C_Minus = Calculate_Value_Asian_Call(S_Minus)
        C = (C_Plus+ C_Minus) / 2
        C_list.append(C)
    
    return C_list

## Simulated value Asian call option
C_list = MC_Asian_Call()
C_list_discount = [np.exp(-r * T) * c for c in C_list]
SE_MC = np.std(C_list_discount) / np.sqrt(R)
Estimate_MC = np.mean(C_list_discount)
Upper_Bound_95_CI = Estimate_MC + norm.ppf(0.975) * SE_MC
Lower_Bound_95_CI = Estimate_MC + norm.ppf(0.025) * SE_MC

## Simulated value Asian call option using antithetic variable technique
C_list_anti = Antithetic_MC_Asian_Call()
C_list_anti_discount = [np.exp(-r * T) * c for c in C_list_anti]
SE_MC_anti = np.std(C_list_anti_discount) / np.sqrt(R)
Estimate_MC_anti = np.mean(C_list_anti_discount)
Upper_Bound_95_CI_anti = Estimate_MC_anti + norm.ppf(0.975) * SE_MC_anti
Lower_Bound_95_CI_anti = Estimate_MC_anti + norm.ppf(0.025) * SE_MC_anti

